In [1]:
!python3 --version

Python 3.10.12


#### fix for could not import -lmza
https://github.com/lucidrains/imagen-pytorch/issues/92 

#### Fix for Can't import datasets AttributeError: partially initialized module 'datasets' has no attribute 'utils' (most likely due to a circular import)

Restart juypyter and kernel


In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
import helper
import time
helper.check_gpu_capacity()

GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 5445 MB
  Used Memory: 17070 MB
GPU 1:
  Total Memory: 23028 MB
  Free Memory: 6217 MB
  Used Memory: 16298 MB
GPU 2:
  Total Memory: 23028 MB
  Free Memory: 6217 MB
  Used Memory: 16298 MB
GPU 3:
  Total Memory: 23028 MB
  Free Memory: 6405 MB
  Used Memory: 16110 MB


In [4]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

print(f"dataset size: {len(dataset)}")

# dataset size: 15011


Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


dataset size: 15011


In [5]:
sample_ds = dataset[randrange(len(dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
Which characters belong to DC or Marvel Universe? Scarlet Witch, Zatanna
--> context

--> response
Zatanna is DC, Scarlet Witch is Marvel
--> category
classification


In [6]:
set(dataset['category'])

{'brainstorming',
 'classification',
 'closed_qa',
 'creative_writing',
 'general_qa',
 'information_extraction',
 'open_qa',
 'summarization'}

In [7]:
# len(dataset.map( ['context']))

In [8]:
# list(filter(lambda x :x != '', list(map(lambda x:((x['category'], len(x['context'])) if len(x['context']) > 5000 else ''), dataset))))

Based on our use case, we want to focus on 'general_qa' and  'information_extraction tasks.,

In [9]:
target_categories = ['closed_qa', 'summarization', 'information_extraction']
filtered_dataset = dataset.filter(lambda x: x['category'] in target_categories)

print(set(filtered_dataset['category']))
filtered_dataset

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-67fe253a0190166e.arrow


{'information_extraction', 'summarization', 'closed_qa'}


Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 4467
})

In [12]:
sample_ds = filtered_dataset[randrange(len(filtered_dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
In which country is Jean Marie Ralph Féthière a politician?
--> context
Jean Marie Ralph Féthière is a Haitian politician. He is a Senator from the north and a member of the ruling PHTK party. On September 23, 2019, while trying to leave Parliament amid a crowd as the government was voting to confirm a new prime minister, Féthière drew a handgun and fired toward the crowd. Chery Dieu-Nalio, a photographer for the Associated Press, suffered injuries to his face from bullet fragments, while a security guard named Leon Leblanc was also injured.
--> response
Haiti
--> category
closed_qa


To instruct tune our model, we need to convert our structured examples into a collection of tasks described via instructions. We define a formatting_function that takes a sample and returns a string with our format instruction.

In [13]:
# def format_instruction(sample):
#     return f"""### Instruction:
#             Use the Input and context below to create an instruction, which could have been used to generate the input using an LLM. 
            
#             ### Input:
#             {sample['response']}
    
#             ### Context:
#             {sample['context'] if sample['context'] != '' else ''}
    
#             ### Response:
#             {sample['instruction']}
#         """

def format_instruction(sample):
    return f"""### Instruction:
            Use the Input and context below to create an instruction, which could have been used to generate the input using an LLM.             
            {sample['context']}
            
            {sample['response']}

            ### Response:
            {sample['instruction']}
        """


Let's test our formatting function on a random example.

In [14]:
from random import randrange

print(format_instruction(filtered_dataset[randrange(len(filtered_dataset))]))

## Do I need to add the context in the format_instruction?????

### Instruction:
            Use the Input and context below to create an instruction, which could have been used to generate the input using an LLM.             
            Historians who address the origins of the American Civil War today agree that the preservation of slavery in the United States was the principal aim of the 11 Southern states (seven states before the onset of the war and four states after the onset) that declared their secession from the United States (the Union) and united to form the Confederate States of America (known as the "Confederacy"). However, while historians in the 21st century agree on the centrality of the conflict over slavery—it was not just "a cause" of the war but "the cause"—they disagree sharply on which aspects of this conflict (ideological, economic, political, or social) were most important, and on the North’s reasons for refusing to allow the Southern states to secede. Proponents of the pseudo-historical Lost Cause ideology have denied that

In [15]:
print(format_instruction(sample_ds))

### Instruction:
            Use the Input and context below to create an instruction, which could have been used to generate the input using an LLM.             
            Jean Marie Ralph Féthière is a Haitian politician. He is a Senator from the north and a member of the ruling PHTK party. On September 23, 2019, while trying to leave Parliament amid a crowd as the government was voting to confirm a new prime minister, Féthière drew a handgun and fired toward the crowd. Chery Dieu-Nalio, a photographer for the Associated Press, suffered injuries to his face from bullet fragments, while a security guard named Leon Leblanc was also injured.
            
            Haiti

            ### Response:
            In which country is Jean Marie Ralph Féthière a politician?
        


In [16]:
shuffled_dataset = filtered_dataset.shuffle(seed=42)
shuffled_dataset = shuffled_dataset.train_test_split(test_size=0.1)

train_dataset = shuffled_dataset['train']
test_dataset = shuffled_dataset['test']

train_eval_dataset = train_dataset.train_test_split(test_size=0.2)
train_dataset = train_eval_dataset['train']
eval_dataset = train_eval_dataset['test']


# Print the sizes of the train, eval, andtest sets
print("Dataset size:", len(dataset))
print("Train size:", len(train_dataset))
print("Eval size:", len(eval_dataset))
print("Test size:", len(test_dataset))

Loading cached shuffled indices for dataset at /home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-be8fe28074092229.arrow


Dataset size: 15011
Train size: 3216
Eval size: 804
Test size: 447


### Instruction-tune Llama 2 using trl and the SFTTrainer

In [17]:
!pip install optimum -q

In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False
helper.check_gpu_capacity()


GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 5445 MB
  Used Memory: 17070 MB
GPU 1:
  Total Memory: 23028 MB
  Free Memory: 6217 MB
  Used Memory: 16298 MB
GPU 2:
  Total Memory: 23028 MB
  Free Memory: 6217 MB
  Used Memory: 16298 MB
GPU 3:
  Total Memory: 23028 MB
  Free Memory: 6405 MB
  Used Memory: 16110 MB


In [19]:
# Hugging Face model id
model_llama_2_7b_id = "lmsys/vicuna-7b-v1.5" # non-gated
# model_id = "meta-llama/Llama-2-7b-hf" # gated

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
torch.cuda.set_device(1)
# Load model and tokenizer
model_llama_2_7b = AutoModelForCausalLM.from_pretrained(model_llama_2_7b_id, 
                                                        quantization_config=bnb_config, 
                                                        use_cache=False, 
                                                        device_map="auto")
# model_llama_2_7b.to_bettertransformer()
tokenizer_llama_2_7b = AutoTokenizer.from_pretrained(model_llama_2_7b_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
sample_ds = filtered_dataset[35]

sample_ds = filtered_dataset[randrange(len(filtered_dataset))]

# prompt = f"""### Instruction:
#             Use the Input and context below to create an instruction, which could have been used to generate the input using an LLM. 
            
#             ### Input:
#             {sample_ds['response']}

#             ### Context:
#             {sample_ds['context'] if sample_ds['context'] != '' else ''}

#             ### Response:
#         """

prompt = f"""### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM.             
            {sample_ds['context']}
            
            {sample_ds['response']}

            ### Response:
        """

input_ids = tokenizer_llama_2_7b(prompt, return_tensors="pt", 
                                 max_length=2000,
                                 truncation=True
                                ).input_ids.cuda()
# with torch.inference_mode():
outputs = model_llama_2_7b.generate(input_ids=input_ids, max_new_tokens=200, 
                                    do_sample=True, top_p=0.9, temperature=0.9
                                   )

print(f"Prompt:\n{prompt}\n")
print(f"Generated instruction:\n{tokenizer_llama_2_7b.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample_ds['instruction']}")

Prompt:
### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM.             
            The Botola Pro (Arabic: البطولة الاحترافية, romanized: al-buṭūla l-iḥtirāfiyya), is a Moroccan professional league for men's association football clubs. At the top of the Moroccan football league system, it is the country's primary football competition. Contested by 16 clubs, it operates on a system of promotion and relegation with the Botola 2.

Seasons run from August to May, with teams playing 30 matches each (playing each team in the league twice, home and away) totaling 240 matches in the season. Most games are played in the afternoons of Saturdays and Sundays, the other games during weekday evenings. It is sponsored by Inwi and thus known as the Botola Pro Inwi. From 2015 to 2019, the league was called Botola Maroc Telecom for sponsorship reasons.
            
            Botola Pro is a moroccan professional le

In [19]:
## From above, we see that the generated response is not okay

In [21]:

model_llama_2_7b.config.pretraining_tp = 1

# tokenizer_llama_2_7b = AutoTokenizer.from_pretrained(model_llama_2_7b_id)
tokenizer_llama_2_7b.pad_token = tokenizer_llama_2_7b.eos_token
tokenizer_llama_2_7b.padding_side = "right"

The SFTTrainer supports a native integration with peft, which makes it super easy to efficiently instruction tune LLMs. We only need to create our LoRAConfig and provide it to the trainer.

In [22]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"



In [24]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)

# prepare model for training
model_llama_2_7b = prepare_model_for_kbit_training(model_llama_2_7b)
model_llama_2_7b = get_peft_model(model_llama_2_7b, peft_config)
# print(print_number_of_trainable_model_parameters(model_llama_2_7b_peft))
model_llama_2_7b.print_trainable_parameters()

trainable params: 33,554,432 || all params: 3,533,967,360 || trainable%: 0.9494833591219133


Before we can start our training we need to define the hyperparameters (TrainingArguments) we want to use.

In [25]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="models/peft-vicuna-int4-instruct-generation-training",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    # save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False, # disable tqdm since with packing values are in correct
    save_total_limit = 2,
    save_strategy="no",
    load_best_model_at_end=True
)


In [26]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model_llama_2_7b,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer_llama_2_7b,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)


In [27]:
helper.check_gpu_capacity()
# del model_llama_2_7b
# del model_llama_2_7b_kbit
helper.free_gpu_memory()
helper.memory_stats()


GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 3185 MB
  Used Memory: 19330 MB
GPU 1:
  Total Memory: 23028 MB
  Free Memory: 4281 MB
  Used Memory: 18234 MB
GPU 2:
  Total Memory: 23028 MB
  Free Memory: 4281 MB
  Used Memory: 18234 MB
GPU 3:
  Total Memory: 23028 MB
  Free Memory: 2949 MB
  Used Memory: 19566 MB
GPU Memory Used: 1.12 GB
GPU Memory Cached: 1.16 GB


In [28]:
%%time

# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()
# After training, access the path of the best checkpoint like this
best_ckpt_path = trainer.state.best_model_checkpoint

Token indices sequence length is longer than the specified maximum sequence length for this model (4886 > 4096). Running this sequence through the model will result in indexing errors
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB (GPU 0; 21.99 GiB total capacity; 3.81 GiB already allocated; 1.15 GiB free; 3.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Test Model and run Inference

In [21]:
if use_flash_attention:
    # unpatch flash attention
    from utils.llama_patch import unplace_flash_attn_with_attn
    unplace_flash_attn_with_attn()

import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

args.output_dir = "llama-7-int4-dolly-filtered"

# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)


NameError: name 'args' is not defined

Let’s load the dataset again with a random sample to try to generate an instruction.

In [33]:
# from datasets import load_dataset
# from random import randrange


# # Load dataset from the hub and get a sample
# dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
# sample = dataset[randrange(len(dataset))]

# prompt = f"""### Instruction:
# Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

# ### Input:
# {sample_ds['response']}

# ### Response:
# """

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
4.1.	UV-Light
Together with oxygen, UV-light can produce a singlet oxygen which can form directly a hydroperoxide from a unsaturated lipid (see Figure 3). Usually during the processing of powders, prevention of UV-light intrusion is being controlled. Furthermore, it depends on the package used, but as well how the consumer stores the product. 
4.2.	Oxygen
Reaction with oxygen can take at various sites in the lipid oxidation chain reaction (see Figure 3). Exclusion of oxygen means mainly flushing inert gases typically nitrogen (or argon) to displace oxygen, packaging in vacuum or in modified atmosphere packaging (MAP). Oxygen scavengers can be employed to completely remove residual oxygen from the system. 
4.3.	Heat
Higher temperatures speed up the lipid oxidation reaction rates. Although the product should be prevented from heat, during production heat is needed to: 
1.	be able to handle ingredients (e.g. melting of fat, best done under nitrogen)
2.	get a product microbiologically stable
3.	spray dry the product
Clearly heat is needed during production but where possible it should be minimized to prevent lipid oxidation. 

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.2)

# print(f"Prompt:\n{sample_ds['response']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
# print(f"Ground truth:\n{sample_ds['instruction']}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Generated instruction:
What are the main factors that cause lipid oxidation in food?
        


### Evaluate the Model Quantitatively (with ROUGE Metric)
Perform inferences for the sample of the test dataset (only 50 instructions and responses to save time). 

In [34]:
dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 15011
})

In [35]:
import pandas as pd


instruction_test = dataset['instruction'][50:100]
response_test = dataset['response'][50:100]

generated_instruction_list = []
ground_truth_list = []

for instruction, response in zip(instruction_test, response_test):
    prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{response}

### Response:
"""

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    # with torch.inference_mode():
    outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.2)
    
    # print(f"Prompt:\n{sample_ds['response']}\n")
    generated_instruction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
    generated_instruction = generated_instruction.split('\n')[0].strip()

    generated_instruction_list.append(generated_instruction)
    ground_truth_list.append(instruction)

zipped_summaries = list(zip(generated_instruction_list, ground_truth_list))
 
df = pd.DataFrame(zipped_summaries, columns = ['generated_instruction_list', 'ground_truth_list'])
df

,generated_instruction_list,ground_truth_list
0,What are the 10 commandments of life?,Think of some family rules to promote a health...
1,Who is Karlon Stark,"In the series A Song of Ice and Fire, who is t..."
2,What is enriched air?,What is enriched air and why would divers dive...
3,What is Casablanca?,What film won the 1943 Oscar as best film
4,Is cold water heavier than hot water?,"which weighs more, cold or hot water?"
5,What is the top speed of a Kia Stinger?,"Given this paragraph, what is the top speed of..."
6,Is it a good idea to have both a cat and a bir...,Write a short paragraph about why you should n...
7,What is the publishing rate of individual stri...,What is your favorite strip from the comic Cal...
8,What are the main aspects that make F1 cars fast?,What makes a formula one car so fast?
9,What is the Key lime pie?,Without quoting directly from the text give me...


In [37]:
for index, row in df[40:50].iterrows():
    print('GI: ', row['generated_instruction_list'])

    print('GT: ',row['ground_truth_list'], '\n')

GI:  What is the difference between Viola toeria and Samphor?
GT:  Identify which instrument is string or percussion: Samphor, Viola toeria 

GI:  What are the things found inside a house?
GT:  If we were playing a game where we had to identify things that can be found inside a house, which of these would we call out: car, chair, table, park, cloud, microwave. 

GI:  What are the different types of beer?
GT:  Classify the following as either dark-colored beers or light colored beers: porter, pilsner, stout, amber, lager 

GI:  
GT:  Give me a bulleted list of the 5 highest mountains in the world and their respective heights in meters 

GI:  • When was Daniel Leavitt born?
GT:  Extract all of the dates mentioned in this paragraph and list them using bullets in the format {Date} - {Description} 

GI:  What products does Apple sell?
GT:  Which products apple sell? 

GI:  What is the moral of the story?
GT:  Write a short story about a person who discovers a hidden room in their house. The

In [38]:
!pip install evaluate==0.4.0 rouge_score==0.1.2 --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Compute ROUGE score for this subset of the data. 

In [39]:
import evaluate

rouge = evaluate.load('rouge')

model_results = rouge.compute(
    predictions=generated_instruction_list,
    references=ground_truth_list[0:len(generated_instruction_list)],
    use_aggregator=True,
    use_stemmer=True,
)

print('MODEL:')
model_results_df = pd.DataFrame(model_results.items(), columns=['Metric', 'Value'])
model_results_df

MODEL:


,Metric,Value
0,rouge1,0.467613
1,rouge2,0.297052
2,rougeL,0.436774
3,rougeLsum,0.434922
